In [35]:
import numpy as np
import pandas as pd
import os
import time
import threading
import tkinter as tk
from tkinter import ttk
from binance import Client


In [36]:
api_key = os.environ.get('API_KEY')
secret_key = os.environ.get('API_KEY_PRIVATE')

# Create connection to Binance
client = Client(api_key, secret_key)

In [37]:
#read BTCUSDT data file
df = pd.read_csv('data/BTCUSDT_data.csv')
#drop unamed and index columns
df.drop(["Unnamed: 0", "index"], axis = 1, inplace = True)

In [38]:
def sma(close, period):
    ma = close.rolling(period).mean()
    return pd.Series(ma)


In [39]:
def ema(close, period):
    """ This function takes closing price and period and returns exponential moving average"""
    close = close.squeeze()
    #calculate Multiplier (smoothing factor)
    multiplier = 2 / (period + 1)
    #calculate Simple Moving Average
    ma = sma(close, period)
    #initialize the Exponential Moving Average list
    ema = [np.NaN] * len(close)

    #iterate over the closing prices
    for index in range(len(close)):
        #check index equals the period calculate EMA
        if index == period:
            ema[index] = close.iloc[index] * multiplier + ma[index - 1] * (1 - multiplier)
        #check index is greater than the period calculate follwing EMA
        if index > period:
            ema[index] = close.iloc[index] * multiplier + ema[index - 1] * (1 - multiplier)

    return pd.Series(ema)

In [40]:
def macd(close, fast_period = 12, slow_period = 26, smoothing = 9):
    """ This function takes in closing price, a period of 12 and 26 and a smoothing value and returns"""

    #calculate MACD using the difference of EMA periods 12 and 26
    macd = ema(close, 12) - ema(close, 26)
    #create series for signal calculation
    empty_values = pd.Series([np.NaN]*slow_period)

    #calculate signal line using smoothing value
    signal_calc_values = macd.iloc[slow_period::].reset_index()
    signal_calc_values.drop(['index'], axis = 1, inplace = True)
    signal = pd.concat([empty_values, ema(signal_calc_values[0], smoothing)])
    signal = pd.Series(signal.reset_index().drop(['index'], axis = 1)[0])

    #calculate histogram
    histogram = macd - signal

    return macd, signal, histogram

    

In [41]:
def ichimoku_cloud(high, low, close, conversion_period = 9, base_period = 26, leading_period = 52):
    """ This function takes in price data for high, low, close and also conversion, base and leading periods 
    returning conversion and base line also leading and lagging spans"""
    #calculate conversion line using high and low prices of last 9 periods
    conversion_line = (high.rolling(conversion_period).max() + low.rolling(conversion_period).min()) / 2
    #calculate base line using high and low prices of last 26 periods
    base_line = (high.rolling(base_period).max() + low.rolling(base_period).min()) / 2

    #calculate leading span using high and low prices using period of 52
    leading_span_a = ((conversion_line + base_line) / 2).shift(base_period)
    leading_span_b = (high.rolling(leading_period).max() + low.rolling(leading_period).min()) / 2
    leading_span_b = leading_span_b.shift(base_period)

    #calculate lagging span by shifting using base period
    lagging_span = close.shift(-base_period)
    
    return conversion_line, base_line, leading_span_a, leading_span_b, lagging_span
    

In [42]:
def calculate_atr(data, period = 14):
    """calculate ATR using high, low, close data and period"""
    
    #separate price data
    high = data['High']
    low = data['Low']
    close = data['Close']

    #create empty series
    atr = pd.Series(0.0, index=data.index)
    #set ATR to closing value
    atr[0] = close[0]

    #for each remaining close data calculate ATR
    for i in range(1, len(close)):
        
        range1 = high[i] - low[i]
        range2 = abs(high[i] - close[i-1])
        range3 = abs(low[i] - close[i-1])
        #using the difference of prices true range takes max value of ranges
        true_range = max(range1, range2, range3)
        atr[i] = ((period - 1) * atr[i-1] + true_range) / period

    return atr

In [43]:
# def calculate_obv(data):
#     close = data['Close']
#     volume = data['Volume']
#     obv = pd.Series(0, index=data.index)
#     obv[0] = volume[0]
#     for i in range(1, len(close)):
#         if close[i] > close[i-1]:
#             obv[i] = obv[i-1] + volume[i]
#         elif close[i] < close[i-1]:
#             obv[i] = obv[i-1] - volume[i]
#         else:
#             obv[i] = obv[i-1]
#     return obv

In [44]:
def calculate_donchian_channels(data, window_size = 20):
    """calculate donchain using closing data and n value of 20 parameters
    returns upper and lower channels and midline"""

    #separate price data
    close = data['Close']

    #calculate upperchannel using max value in window
    upper_channel = close.rolling(window=window_size).max()
    #calculate lowerchannel using min value in window
    lower_channel = close.rolling(window=window_size).min()
    #calculate midline using upper and lower channel /2
    mid_line = (upper_channel + lower_channel) / 2
    
    return upper_channel, lower_channel, mid_line


In [45]:
import numpy as np
import pandas as pd

def rsi(close, number_of_periods=14):
    """ calculate RSI using closing price and number of periods
    returns RSI"""

    close = pd.Series(close.squeeze())  # Ensure input is a pandas Series

    deltas = close.diff()
    up, down = deltas.copy(), deltas.copy()

    up[up < 0] = 0
    down[down > 0] = 0
    down = down.abs()

    avg_gain = up.rolling(window=number_of_periods, min_periods=number_of_periods).mean()
    avg_loss = down.rolling(window=number_of_periods, min_periods=number_of_periods).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi.values


In [46]:
def bollinger_bands(close, window_size=20):
    """calculate bollinger bands using closing price parameters closing price, window size
    returns upper, lower and rolling mean for bollinger bands"""
    #calculate rolling mean of closing price
    rolling_mean = close.rolling(window_size).mean()
    #calculate standard deviation of closing price
    rolling_std = close.rolling(window_size).std()

    #calculate upper band using rolling mean and rolling standard deviation
    upper_band = rolling_mean + (rolling_std * 2)
    #calculate lower band using rolling mean and rolling standard deviation
    lower_band = rolling_mean - (rolling_std * 2)

    return upper_band, rolling_mean, lower_band

In [47]:
def fibonacci_retracement(close, open):
    """calculate fibonacci retracement using close and open price
    returns fibonacci retracement in separate dataframe"""
    #fibonacci levels
    fib_levels = [0, 0.236, 0.382, 0.5, 0.618, 0.764, 1.0]

    diff = close - open
    #list comprehension
    levels = np.array([close - level * diff for level in fib_levels])
    df = pd.DataFrame(levels.T, columns=[f"Fibonacci {int(level*100)}%" for level in fib_levels])
    return df

In [48]:
def use_macd(df):
    df['macd'], df['signal'], df['histogram'] = macd(df.Close)
    df['prev_histogram'] = df['histogram'].shift(1)
    # trading_df = df[(df.Time >= '2022-12-01 00:00:00') & (df.Time <= '2023-02-01 23:59:00')]
    df['buy_sell'] = 0

    buy_condition_1 = (df['prev_histogram'] < 0) & (df['histogram'] > 0) 
    buy_condition_2 = (df['macd'] < 0) & (df['signal'] < 0)
    
    df.loc[buy_condition_1 & buy_condition_2, 'buy_sell'] = 1

    sell_signal_1 = (df['prev_histogram'] > 0) & (df['histogram'] < 0) 
    sell_signal_2 = (df['macd'] > 0) & (df['signal'] > 0)

    df.loc[sell_signal_1 & sell_signal_2, 'buy_sell'] = -1

    #Clean up df
    df.drop(['macd', 'signal', 'histogram', 'prev_histogram'], axis = 1, inplace = True)

    return df

In [49]:
# Testing macd_function
macd_test = use_macd(df)
macd_test[(macd_test['buy_sell'] == 1) | (macd_test['buy_sell'] == -1)]

,Name,Time,Open,High,Low,Close,Volume,buy_sell
54,BTCUSDT,2022-11-30 20:54:00,17044.56,17053.00,17043.05,17046.15,154.12992,1
68,BTCUSDT,2022-11-30 21:08:00,17074.85,17077.27,17054.49,17060.11,252.28819,-1
88,BTCUSDT,2022-11-30 21:28:00,17036.55,17049.21,17035.78,17047.69,152.91954,1
123,BTCUSDT,2022-11-30 22:03:00,17094.58,17100.41,17092.00,17098.90,134.64526,-1
151,BTCUSDT,2022-11-30 22:31:00,17102.90,17106.74,17096.12,17096.95,199.00515,-1
...,...,...,...,...,...,...,...,...
90888,BTCUSDT,2023-02-01 22:48:00,23712.21,23722.00,23700.01,23715.51,89.66899,1
90909,BTCUSDT,2023-02-01 23:09:00,23711.72,23716.91,23699.00,23714.58,244.42854,-1
90926,BTCUSDT,2023-02-01 23:26:00,23754.84,23755.81,23743.31,23744.74,125.11862,-1
90951,BTCUSDT,2023-02-01 23:51:00,23701.50,23714.42,23701.22,23708.88,126.76657,1


In [50]:
def use_sma(df):
    df['sma_50'] = sma(df.Close, 50)
    df['sma_200'] = sma(df.Close, 200)
    df['sma_diff'] = df['sma_50'] - df['sma_200']
    df['prev_sma_diff'] = df['sma_diff'].shift(1)
    trading_df = df[(df.Time >= '2022-12-01 00:00:00') & (df.Time <= '2023-02-01 23:59:00')]
    trading_df['buy_sell'] = 0

    buy_condition = (trading_df['sma_diff'] > 0) & (trading_df['prev_sma_diff'] < 0)
    sell_condition = (trading_df['sma_diff'] < 0) & (trading_df['prev_sma_diff'] > 0)

    trading_df.loc[buy_condition, 'buy_sell'] = 1
    trading_df.loc[sell_condition, 'buy_sell'] = -1

    # Clean up df
    df.drop(['sma_50', 'sma_200', 'sma_diff', 'prev_sma_diff'], axis = 1, inplace = True)

    return trading_df

In [51]:
# Test sma
sma_test = use_sma(df)
sma_test[(sma_test['buy_sell'] == 1) | (sma_test['buy_sell'] == -1)]

C:\Users\nolan\AppData\Local\Temp\ipykernel_22448\1969192233.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trading_df['buy_sell'] = 0


,Name,Time,Open,High,Low,Close,Volume,buy_sell,sma_50,sma_200,sma_diff,prev_sma_diff
329,BTCUSDT,2022-12-01 01:29:00,17122.48,17127.38,17119.81,17123.66,165.05873,-1,17149.3450,17149.40965,-0.06465,0.89295
459,BTCUSDT,2022-12-01 03:39:00,17146.15,17149.55,17142.65,17147.13,98.63403,1,17141.3966,17141.03900,0.35760,-0.20315
575,BTCUSDT,2022-12-01 05:35:00,17110.78,17116.36,17104.74,17108.92,313.47280,-1,17139.2864,17139.87275,-0.58635,0.08915
797,BTCUSDT,2022-12-01 09:17:00,17098.74,17100.55,17093.11,17098.56,104.11771,1,17102.1006,17101.95570,0.14490,-0.19880
824,BTCUSDT,2022-12-01 09:44:00,17105.88,17110.91,17100.91,17102.47,214.30144,-1,17100.2578,17100.55335,-0.29555,0.10425
...,...,...,...,...,...,...,...,...,...,...,...,...
89768,BTCUSDT,2023-02-01 04:08:00,23136.53,23140.52,23134.97,23137.58,143.04228,1,23103.0784,23102.60380,0.47460,-0.33170
89902,BTCUSDT,2023-02-01 06:22:00,23088.80,23102.00,23082.32,23098.91,167.89105,-1,23123.4918,23123.92500,-0.43320,0.61005
90200,BTCUSDT,2023-02-01 11:20:00,23032.00,23034.81,23030.00,23033.63,69.25971,1,22998.0552,22998.01845,0.03675,-1.13630
90438,BTCUSDT,2023-02-01 15:18:00,23047.30,23062.60,23042.92,23056.63,218.82545,-1,23087.6694,23088.26775,-0.59835,0.33790


In [52]:
def use_rsi(df, overbought_thresh = 70, oversold_thresh = 30):
    df['rsi'] = rsi(df.Close)
    df['prev_rsi'] = df['rsi'].shift(1)

    trading_df = df[(df.Time >= '2022-12-01 00:00:00') & (df.Time <= '2023-02-01 23:59:00')]
    trading_df['buy_sell'] = 0

    buy_condition = (trading_df['rsi'] < oversold_thresh) & (trading_df['prev_rsi'] >= oversold_thresh)
    sell_condition = (trading_df['rsi'] > overbought_thresh) & (trading_df['prev_rsi'] <= overbought_thresh)

    trading_df.loc[buy_condition, 'buy_sell'] = 1
    trading_df.loc[sell_condition, 'buy_sell'] = -1
    # Clean up df
    df.drop(['rsi', 'prev_rsi'], axis = 1, inplace = True)
    return trading_df

In [53]:
# Test rsi
rsi_test = use_rsi(df)
rsi_test[(rsi_test['buy_sell'] == 1) | (rsi_test['buy_sell'] == -1)]

C:\Users\nolan\AppData\Local\Temp\ipykernel_22448\2649809901.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trading_df['buy_sell'] = 0


,Name,Time,Open,High,Low,Close,Volume,buy_sell,rsi,prev_rsi
245,BTCUSDT,2022-12-01 00:05:00,17165.44,17167.28,17152.00,17152.09,174.63185,1,26.070114,40.450502
256,BTCUSDT,2022-12-01 00:16:00,17194.13,17220.00,17192.32,17217.68,509.72010,-1,73.666107,59.290832
276,BTCUSDT,2022-12-01 00:36:00,17174.15,17181.77,17169.04,17174.49,154.11843,1,28.261397,36.222439
279,BTCUSDT,2022-12-01 00:39:00,17171.78,17177.37,17165.48,17166.70,145.60302,1,22.689882,32.048735
331,BTCUSDT,2022-12-01 01:31:00,17122.35,17127.55,17120.76,17124.37,134.67023,1,26.103319,31.504677
...,...,...,...,...,...,...,...,...,...,...
90884,BTCUSDT,2023-02-01 22:44:00,23669.87,23671.32,23654.83,23657.88,183.10401,1,27.199481,34.879436
90906,BTCUSDT,2023-02-01 23:06:00,23759.39,23765.65,23754.91,23761.07,133.90001,-1,73.079391,68.638427
90922,BTCUSDT,2023-02-01 23:22:00,23771.79,23786.38,23768.80,23774.05,225.70118,-1,79.857979,60.671463
90938,BTCUSDT,2023-02-01 23:38:00,23718.81,23724.41,23702.89,23706.43,218.79542,1,27.907306,33.956729


In [54]:
def use_ichimoku(df):
    df['conversion_line'], df['base_line'], df['leading_span_a'], df['leading_span_b'], df['lagging_span'] = ichimoku_cloud(df.High, df.Low, df.Close)
    df['conversion_base_diff'] = df['conversion_line'] - df['base_line']
    df['prev_diff'] = df['conversion_base_diff'].shift(1)
    
    trading_df = df[(df.Time >= '2022-12-01 00:00:00') & (df.Time <= '2023-02-01 23:59:00')]
    trading_df['buy_sell'] = 0

    buy_condition_1 = trading_df.Close >= np.maximum(trading_df['leading_span_a'], trading_df['leading_span_b'])
    buy_condition_2 = (trading_df['conversion_base_diff'] > 0) & (trading_df['prev_diff'] < 0)
    trading_df.loc[buy_condition_1 & buy_condition_2, 'buy_sell'] = 1

    sell_condition_1 = trading_df.Close < np.minimum(trading_df['leading_span_a'], trading_df['leading_span_b'])
    sell_condition_2 = (trading_df['conversion_base_diff'] < 0) & (trading_df['prev_diff'] > 0)
    trading_df.loc[sell_condition_1 & sell_condition_2, 'buy_sell'] = -1

    # Clean up df
    df.drop(['conversion_line', 'base_line', 'leading_span_a', 'leading_span_b', 'lagging_span', 'conversion_base_diff', 'prev_diff'], axis = 1, inplace = True)
    return trading_df

In [55]:
#Test ichimoku
ichimoku_test = use_ichimoku(df)
ichimoku_test[(ichimoku_test['buy_sell'] == 1) | (ichimoku_test['buy_sell'] == -1)]

C:\Users\nolan\AppData\Local\Temp\ipykernel_22448\1436316360.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trading_df['buy_sell'] = 0


,Name,Time,Open,High,Low,Close,Volume,buy_sell,conversion_line,base_line,leading_span_a,leading_span_b,lagging_span,conversion_base_diff,prev_diff
256,BTCUSDT,2022-12-01 00:16:00,17194.13,17220.00,17192.32,17217.68,509.72010,1,17186.855,17176.675,17192.7175,17168.605,17165.03,10.180,-9.490
315,BTCUSDT,2022-12-01 01:15:00,17145.44,17151.59,17144.92,17149.36,90.33405,-1,17146.200,17146.485,17174.3250,17184.820,17129.16,-0.285,2.855
320,BTCUSDT,2022-12-01 01:20:00,17142.10,17144.19,17135.84,17140.24,123.01189,-1,17146.535,17152.195,17158.8025,17179.470,17134.29,-5.660,0.050
354,BTCUSDT,2022-12-01 01:54:00,17129.61,17131.00,17122.94,17124.62,126.71598,-1,17130.435,17130.920,17132.5125,17142.680,17121.70,-0.485,6.735
381,BTCUSDT,2022-12-01 02:21:00,17122.44,17123.22,17109.00,17113.50,195.92541,-1,17124.300,17125.995,17126.2650,17137.845,17129.72,-1.695,3.870
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90674,BTCUSDT,2023-02-01 19:14:00,23097.25,23100.00,23066.44,23082.05,272.82793,1,23059.000,23057.755,23056.0300,23033.915,23166.34,1.245,-2.810
90677,BTCUSDT,2023-02-01 19:17:00,23045.31,23048.17,23001.00,23020.61,478.82619,-1,23055.500,23057.755,23056.4150,23033.915,23271.54,-2.255,17.650
90746,BTCUSDT,2023-02-01 20:26:00,23428.94,23454.25,23423.53,23438.12,550.31760,1,23398.125,23393.130,23296.5875,23127.500,23487.55,4.995,-15.575
90784,BTCUSDT,2023-02-01 21:04:00,23562.92,23575.00,23535.27,23570.40,321.63149,1,23558.575,23551.000,23541.8575,23461.075,23745.72,7.575,-9.310


In [56]:
def use_donchian_channel(df):
    df['upper'], df['lower'], df['mid'] = calculate_donchian_channels(df)
    trading_df = df[(df.Time >= '2022-12-01 00:00:00') & (df.Time <= '2023-02-01 23:59:00')]
    trading_df['buy_sell'] = 0

    buy_condition = trading_df.Close >= trading_df.upper
    trading_df.loc[buy_condition, 'buy_sell'] = 1

    sell_condition = trading_df.Close <= trading_df.lower
    trading_df.loc[sell_condition, 'buy_sell'] = -1
    
    df.drop(['upper', 'lower', 'mid'], axis = 1, inplace = True)
    return trading_df
    

In [57]:
donchian_test = use_donchian_channel(df)
donchian_test[(donchian_test['buy_sell'] == 1) | (donchian_test['buy_sell'] == -1)]


C:\Users\nolan\AppData\Local\Temp\ipykernel_22448\3765946724.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trading_df['buy_sell'] = 0


,Name,Time,Open,High,Low,Close,Volume,buy_sell,upper,lower,mid
242,BTCUSDT,2022-12-01 00:02:00,17174.76,17183.76,17157.18,17161.26,226.98035,-1,17222.12,17161.26,17191.690
245,BTCUSDT,2022-12-01 00:05:00,17165.44,17167.28,17152.00,17152.09,174.63185,-1,17203.21,17152.09,17177.650
246,BTCUSDT,2022-12-01 00:06:00,17152.91,17155.63,17139.33,17147.26,227.17760,-1,17203.21,17147.26,17175.235
254,BTCUSDT,2022-12-01 00:14:00,17178.46,17190.54,17177.30,17188.29,164.81992,1,17188.29,17147.26,17167.775
255,BTCUSDT,2022-12-01 00:15:00,17188.85,17196.61,17179.88,17195.25,256.69111,1,17195.25,17147.26,17171.255
...,...,...,...,...,...,...,...,...,...,...,...
90939,BTCUSDT,2023-02-01 23:39:00,23706.42,23710.76,23693.66,23697.40,270.47691,-1,23774.05,23697.40,23735.725
90947,BTCUSDT,2023-02-01 23:47:00,23700.04,23700.77,23682.94,23695.34,330.61525,-1,23763.11,23695.34,23729.225
90948,BTCUSDT,2023-02-01 23:48:00,23695.34,23703.68,23690.04,23694.87,143.62819,-1,23763.11,23694.87,23728.990
90957,BTCUSDT,2023-02-01 23:57:00,23713.36,23726.83,23712.72,23722.29,170.78623,1,23722.29,23694.87,23708.580


In [58]:
def get_data(symbol, interval, lookback):
    data = []
    while data == []:
        data = client.get_historical_klines(symbol,
                                      interval,
                                      str(lookback) + ' minute ago UTC')
    df = pd.DataFrame(data)
    df = df.iloc[:,:6]
    df.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    df.insert(loc=0, column='Name', value=symbol)
    df.iloc[:,1:] = df.iloc[:,1:].astype(float)
    df.Time = pd.to_datetime(df.Time, unit = 'ms')
    # df.set_index('Time', inplace = True)
    
    return df

In [59]:
def get_ta(df, indicator = macd):
    df = use_macd(df)
    return df

In [60]:
def buy(trading_data, buy_type, fiat_amount):  
    trading_data = trading_data[(trading_data['buy_sell'] == 1) | (trading_data['buy_sell'] == -1)]
    if buy_type == 'all':
        position = False
        coin = 0
        for idx, row in trading_data.iterrows():
            if (row['buy_sell'] == 1) & (position == False):
                position = True
                coin = fiat_amount / row['Close']
                #print("Bought {} with {}".format(coin, fiat_amount))
                fiat_amount = 0
                
            elif (row['buy_sell'] == -1) & (position == True):
                fiat_amount = coin * row['Close']
                #print("Sold {} for {}".format(coin, fiat_amount))
                coin = 0
                position = False
        fiat_amount += coin * trading_data.iloc[-1, 5]
        coin = 0
    
    if buy_type == 'consec':
        position = False
        fiat_buy = fiat_amount / 4
        coin = 0
        for idx, row in trading_data.iterrows():
            if (row['buy_sell'] == 1) & (fiat_amount > fiat_buy):
                position = True
                coin += fiat_buy / row['Close']
                #print("Bought {} with {}".format(coin, fiat_amount))
                fiat_amount -= fiat_buy
            elif (row['buy_sell'] == 1) & (fiat_amount < fiat_buy) & (fiat_amount > 0):
                position = True
                coin += fiat_amount / row['Close']
                #print("Bought {} with {}".format(coin, fiat_amount))
                fiat_amount = 0 
            elif (row['buy_sell'] == -1) & (position == True):
                fiat_amount += coin * row['Close']
                #print("Sold {} for {}".format(coin, fiat_amount))
                coin = 0
                fiat_buy = fiat_amount / 4
                position = False
        fiat_amount += coin * trading_data.iloc[-1, 5]
        coin = 0
    return fiat_amount

In [61]:
def backtest(indicator, data = df, buy_type='all', fiat_amount = 0 ):
    # while True:
        # try:
        #     fiat_amount = float(input("Please enter your fiat value: "))
        # except ValueError:
        #     #user input not valid
        #     print("please enter a valid number")
        #     continue
        # else:
    if indicator == 'macd': 
        trading_data = use_macd(df)
        fiat_amount = buy(trading_data, buy_type=buy_type, fiat_amount=fiat_amount)
    
    elif indicator == 'rsi':
        trading_data = use_rsi(df)
        fiat_amount = buy(trading_data, buy_type=buy_type, fiat_amount=fiat_amount)

    elif (indicator == 'ma') | (indicator == 'sma'):
        trading_data = use_sma(df)
        fiat_amount = buy(trading_data, buy_type=buy_type, fiat_amount=fiat_amount)    

    return fiat_amount
        #successful input break loop and continue
    # break

    

In [85]:
def trade(token, indicator,stop_event, fiat_amount = 0,  buy_type = 'all'):
    df = get_data(token, Client.KLINE_INTERVAL_1MINUTE, '500')
    df = get_ta(df)
    time.sleep(60)
    if buy_type == 'all':
        position = False
        coin = 0
        while not stop_event.is_set():
            file_mod_time = os.stat('output.txt').st_mtime
            df = pd.concat([df, get_data('BTCUSDT', Client.KLINE_INTERVAL_1MINUTE, '1')], ignore_index=True)
            df = get_ta(df)
            if (df.iloc[-1,-1] == 1) & (position == False):
                position = True
                coin = fiat_amount / df.iloc[-1, 5]
                with open("output.txt", "a+") as file:
                    file.write("Bought {} with {}".format(coin, fiat_amount) +"\n")
                    file.write("File modified after {}".format(time.time() - file_mod_time))
                fiat_amount = 0
                
            elif (df.iloc[-1,-1] == -1) & (position == True):
                fiat_amount = coin * df.iloc[-1, 5]
                with open("output.txt", "a+") as file:
                    file.write("Sold {} for {}".format(coin, fiat_amount) +"\n")
                    file.write("File modified after {}".format(time.time() - file_mod_time))
                coin = 0
                position = False
            elif (position == False):
                with open("output.txt", "a+") as file:
                    file.write("Waiting for Buy signal..." +"\n" )
                    file.write("File modified after {}".format(time.time() - file_mod_time))
            elif (position == True):
                with open("output.txt", "a+") as file:
                    file.write("Waiting for Sell signal..." +"\n")
                    file.write("File modified after {}".format(time.time() - file_mod_time))
            time.sleep(60)

    if buy_type == 'consec':
        position = False
        coin = 0
        while not stop_event.is_set():
            df = pd.concat([df, get_data('BTCUSDT', Client.KLINE_INTERVAL_1MINUTE, '1')], ignore_index=True)
            df = get_ta(df)
            fiat_buy = fiat_amount / 4
            if (df.iloc[-1,-1] == 1) & (fiat_amount > fiat_buy):
                position = True
                coin += fiat_buy / df.iloc[-1, 5]
                print("Bought {} with {}".format(coin, fiat_amount))
                fiat_amount -= fiat_buy
            elif (df.iloc[-1,-1] == 1) & (fiat_amount < fiat_buy) & (fiat_amount > 0):
                position = True
                coin += fiat_amount / df.iloc[-1, 5]
                print("Bought {} with {}".format(coin, fiat_amount))
                fiat_amount = 0 
            elif (df.iloc[-1,-1] == -1) & (position == True):
                fiat_amount += coin * df.iloc[-1, 5]
                print("Sold {} for {}".format(coin, fiat_amount))
                coin = 0
                fiat_buy = fiat_amount / 4
                position = False
            time.sleep(60)

   

In [63]:
#trade(indicator = 'sma', buy_type='consec')

In [64]:
result = {'BTC': [],
               'ETH': [],
               'APE': [],
               'CHZ': [],
               'DOG': [],
               'LIN': [],
               'MAN': [],
               'UNI': [],}

files = ['BTCUSDT_data.csv', 'ETHUSDT_data.csv', 'APEUSDT_data.csv', 'CHZUSDT_data.csv', 'DOGEUSDT_data.csv', 'LINKUSDT_data.csv', 'MANAUSDT_data.csv', 'UNIUSDT_data.csv']

#for file in files:
#    df = pd.read_csv('data/'+file)
#    result[file[0:3]].append(trade(indicator = 'rsi', buy_type='all'))
#    result[file[0:3]].append(trade(indicator = 'rsi', buy_type='consec'))
    

In [65]:
#result_macd

Gerald's User Input Prompt to Select the two TA

In [86]:
indicator_functions = {
    'EMA': lambda data: ema(data, 14),
    'MACD': macd,
    'Ichimoku Cloud': ichimoku_cloud,
    'ATR': calculate_atr,
    'Donchian Channels': calculate_donchian_channels,
    'RSI': rsi,
    'Bollinger Bands': bollinger_bands,
    # 'Fibonacci Retracement': fibonacci_retracement,
}

token_list = {
    'BTC': 'BTCUSDT',
    'ETH': 'ETHUSDT',
    'APE': 'APEUSDT',
    'CHZ': 'CHZUSDT',
    'DOG': 'DOGEUSDT',
    'LIN': 'LINKUSDT',
    'MAN': 'MANAUSDT',
    'UNI': 'UNIUSDT',
}


tokens = list(token_list.keys())

def live_trade():
    indicator1 = selected_indicator1.get()
    # indicator2 = selected_indicator2.get()
    fiat_value = float(fiat_value_entry.get())
    token = selected_token.get()

    if not token:
        print("Error: Please select a token.")
        return
    window.destroy()
    with open("output.txt", "w") as file:
            file.write("Initializing..." + "\n")
    
    stop_event = threading.Event()
    t = threading.Thread(target=trade, args=(token_list[token], indicator1.lower(),stop_event, fiat_value, 'all'))
    print("Start Threading")
    t.start()
    new_window = tk.Tk()
    new_window.title("Trading Process")
    
    output_text = tk.Text(new_window)
    output_text.insert(tk.END, 'Initializing...')
    output_text.pack()
    output_text.config(state=tk.DISABLED)
    # trade(token_list[token], indicator=indicator1.lower(), fiat_amount=fiat_value)

    #Multithreading
    update_tk(output_text,new_window)
    new_window.protocol("WM_DELETE_WINDOW", lambda: stop_worker(stop_event,new_window))

    
    new_window.mainloop()

data_files = {
    'BTC': 'data/BTCUSDT_data.csv',
    'ETH': 'data/ETHUSDT_data.csv',
    'APE': 'data/APEUSDT_data.csv',
    'CHZ': 'data/CHZUSDT_data.csv',
    'DOG': 'data/DOGEUSDT_data.csv',
    'LIN': 'data/LINKUSDT_data.csv',
    'MAN': 'data/MANAUSDT_data.csv',
    'UNI': 'data/UNIUSDT_data.csv',
}

dataframes = {}
for token, file in data_files.items():
    df = pd.read_csv(file)
    df.drop(["Unnamed: 0", "index"], axis=1, inplace=True)
    dataframes[token] = df

def submit():
    indicator1 = selected_indicator1.get()
    indicator2 = selected_indicator2.get()
    fiat_value = float(fiat_value_entry.get())
    token = selected_token.get()

    if not token:
        print("Error: Please select a token.")
        return
    window.destroy()
    new_window = tk.Tk()
    new_window.title("Trade Result")
    data = dataframes[token]
    # Calculate the outcomes for the selected indicators
    outcome1 = backtest(indicator=indicator1.lower(), data=data, fiat_amount=fiat_value)
    outcome2 = backtest(indicator=indicator2.lower(), data=data, fiat_amount=fiat_value)
    output_text = tk.Text(new_window, height=10, width=50)
    output_text.pack()
    output_text.insert(tk.END, f"Selected Indicators:\n")
    output_text.insert(tk.END, f"Indicator 1: {indicator1}, Outcome: {outcome1}\n")
    output_text.insert(tk.END, f"Indicator 2: {indicator2}, Outcome: {outcome2}\n")
    output_text.insert(tk.END, f"Fiat Value: {fiat_value}, Token: {token}\n")


def update_tk(output_text, window):
    with open("output.txt", "r") as file:
        content = file.read()
    output_text.config(state=tk.NORMAL)
    output_text.delete('1.0', tk.END)
    output_text.insert(tk.END, content)
    output_text.config(state=tk.DISABLED) 
    print('Screen refreshed')
    window.after(61500, lambda: update_tk(output_text, window))

def stop_worker(stop_event, window):
    stop_event.set()
    window.destroy()





# Create the tkinter window
window = tk.Tk()
window.title("Select Technical Analysis Indicators, Fiat Value, and Token")

# Create the labels and dropdowns for the two indicators
label1 = ttk.Label(window, text="Select Indicator 1:")
label1.grid(column=0, row=0)
selected_indicator1 = tk.StringVar()
dropdown1 = ttk.Combobox(window, textvariable=selected_indicator1, values=list(indicator_functions.keys()))
dropdown1.grid(column=1, row=0)

label2 = ttk.Label(window, text="Select Indicator 2 (Backtest only):")
label2.grid(column=0, row=1)
selected_indicator2 = tk.StringVar()
dropdown2 = ttk.Combobox(window, textvariable=selected_indicator2, values=list(indicator_functions.keys()))
dropdown2.grid(column=1, row=1)

# Create the labels and entry for the fiat value
label3 = ttk.Label(window, text="Enter Fiat Value:")
label3.grid(column=0, row=2)
fiat_value_entry = ttk.Entry(window)
fiat_value_entry.grid(column=1, row=2)

#Create the labels and dropdown for the token
label4 = ttk.Label(window, text="Select Token:")
label4.grid(column=0, row=3)
selected_token = tk.StringVar()
dropdown3 = ttk.Combobox(window, textvariable=selected_token, values=tokens)
dropdown3.grid(column=1, row=3)

#Create a submit button
submit_button = ttk.Button(window, text="Trade", command=live_trade)
submit_button.grid(column=1, row=4)
submit_button = ttk.Button(window, text="Backtest", command=submit)
submit_button.grid(column=0, row=4)

# Start the tkinter main loop
window.mainloop()


Start Threading
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshed
Screen refreshe

# Backtest

In [33]:
df = pd.read_csv('./year_data/ETHUSDT_full.csv')

In [34]:
files = ['BTCUSDT_full.csv', 'ETHUSDT_full.csv', 'APEUSDT_full.csv', 'CHZUSDT_full.csv', 'DOGEUSDT_full.csv', 'LINKUSDT_full.csv', 'MANAUSDT_full.csv', 'UNIUSDT_full.csv']
indicators = [use_macd, use_rsi, use_sma, use_donchian_channel, use_ichimoku]
result_df = pd.DataFrame(index=['BTC','ETH','APE','CHZ','DOG','LIN','MAN','UNI'],columns=['use_macd & use_rsi', 'use_macd & use_sma', 'use_rsi & use_sma'])
for file in files:
    df = pd.read_csv("./year_data/{}".format(file))
    for i in range(len(indicators)):
        for j in range(i + 1,len(indicators)):
            print('Testing:{} & {}'.format(indicators[i].__name__, indicators[j].__name__))
            trading_frame_1 = indicators[i](df)
            trading_frame_2 = indicators[j](df)
            trading_data = df.copy()
            trading_data['buy_sell_1'] = trading_frame_1.iloc[:,-1]
            trading_data['buy_sell_2'] = trading_frame_2.iloc[:,-1]
            
            position = False
            coin = 0
            fiat_amount = 1000
            for idx, row in trading_data.iterrows():
                if (row['buy_sell_1'] == 1) & (row['buy_sell_2'] == 1) & (position == False):
                    position = True
                    coin = fiat_amount / row['Close']
                    print("Bought {} with {}".format(coin, fiat_amount))
                    fiat_amount = 0
                    
                elif ((row['buy_sell_1'] == -1) | (row['buy_sell_2'] == -1)) & (position == True):
                    fiat_amount = coin * row['Close']
                    print("Sold {} for {}".format(coin, fiat_amount))
                    coin = 0
                    position = False
            fiat_amount += coin * trading_data.iloc[-1, 5]
            coin = 0
            
            result_df.loc[file[0:3], '{} & {}'.format(indicators[i].__name__, indicators[j].__name__) ] = fiat_amount



Testing:use_macd & use_rsi


KeyboardInterrupt: 

In [ ]:
result_df